In [1]:
import spacy
import torch
import editdistance
import datetime

from spacy.tokens import Doc, Token, Span
from spacy.vocab import Vocab

from transformers import AutoModelWithLMHead, AutoTokenizer

In [2]:
nlp = spacy.load("en_core_web_sm")

# Define a custom component
def custom_component(doc):
    # Print the doc's length
    print("Doc length:", len(doc))
    # Return the doc object
    return doc

# Add the component first in the pipeline
nlp.add_pipe(custom_component, first=True)

# Print the pipeline component names
print("Pipeline:", nlp.pipe_names)

doc = nlp("this is a small test")
doc[1].orth

Pipeline: ['custom_component', 'tagger', 'parser', 'ner']
Doc length: 5


3411606890003347522

In [3]:
any([True, False])

True

## Identifying OOV

In [48]:
class spellChecker(object):
    """Class object for Out Of Vocabulary(OOV) corrections 
    """
    name = "contextual spellchecker"

    def __init__(self, debug=False):
        self.nlp = spacy.load(
            "en_core_web_sm", disable=["tagger", "parser"]
        )  # using default tokeniser with NER
        with open("./data/vocab.txt") as f:
            # if want to remove '[unusedXX]' from vocab
            # words = [line.rstrip() for line in f if not line.startswith('[unused')]
            words = [line.rstrip() for line in f]
        self.vocab = Vocab(strings=words)
        self.BertTokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
        self.BertModel = AutoModelWithLMHead.from_pretrained("bert-base-cased")
        self.mask = self.BertTokenizer.mask_token
        self.debug = debug
        if not Doc.has_extension('contextual_spellCheck'):
            Doc.set_extension('contextual_spellCheck', default=True)
            Doc.set_extension('performed_spellCheck', default=False)

            # {originalToken-1:[suggestedToken-1,suggestedToken-2,..],
            #  originalToken-2:[...]}
            Doc.set_extension('suggestions_spellCheck', getter=self.doc_suggestions_spellCheck)
            Doc.set_extension('outcome_spellCheck', default="")
            Doc.set_extension('score_spellCheck', default=None)

            Span.set_extension('get_has_spellCheck', getter=self.span_require_spellCheck)
            Span.set_extension('score_spellCheck', getter=self.span_score_spellCheck)

            Token.set_extension('get_require_spellCheck', getter=self.token_require_spellCheck)
            Token.set_extension('get_suggestion_spellCheck', getter=self.token_suggestion_spellCheck)
            Token.set_extension('score_spellCheck', getter=self.token_score_spellCheck)
    
    def token_require_spellCheck(self,token):
        """Getter for Token attributes. 
        @Returns True if the token requires spellCheck
        """
        return any([token.orth == suggestion.orth 
                    for suggestion in token.doc._.suggestions_spellCheck.keys()])
    
    def token_suggestion_spellCheck(self,token):
        """Getter for Token attributes. 
        @Returns    [] or List['suggestion-1','suggestion-1',...] 
                    
        """
        for suggestion in token.doc._.suggestions_spellCheck.keys():
            if token.orth == suggestion.orth:
                return token.doc._.suggestions_spellCheck[token]
        return []
    
    def token_score_spellCheck(self, token):
        """Getter for Token attributes. 
        @Returns    [] or List[('suggestion-1',score-1), ('suggestion-1',score-2), ...] 
                    
        """
        for suggestion in token.doc._.score_spellCheck.keys():
            if token.orth == suggestion.orth:
                return [token.doc._.score_spellCheck[token]]
        return []

        
    def span_score_spellCheck(self, span):
        return [{token:self.token_score_spellCheck(token)} for token in span]
        
    
    def span_require_spellCheck(self,span):
        """Getter for Token attributes. 
        @Returns True if the span requires spellCheck
        """
        return any([self.token_require_spellCheck(token) for token in span])
    
    def doc_suggestions_spellCheck(self,doc):
        response={}
        for token in doc._.score_spellCheck:
            if token not in response:
                response[token]=[]
            for suggestion_score in doc._.score_spellCheck[token]:
                response[token].append(suggestion_score[0])       
        return response

    def check(self, query=""):
        """Complete pipeline which returns update query

        Keyword Arguments:
            query {str} -- User query for which spell checking to be done (default: {''})

        Returns:
            {str} -- returns updated query with spelling corrections (if any)
        """
        if type(query) != str and len(query) == 0:
            print("Invalid query, expected non empty `str` but passed", query)

        modelLodaded = datetime.datetime.now()
        misspellTokens, doc = self.misspellIdentify(query)
        modelLoadTime = timeLog("Misspell identification: ",modelLodaded)
        if len(misspellTokens) > 0:
            candidate = self.candidateGenerator(doc, misspellTokens, query=query)
            answer = self.candidateRanking(candidate)
            updatedQuery = ""
            for i in doc:
                if i in misspellTokens:
                    updatedQuery += answer[i] + i.whitespace_
                else:
                    updatedQuery += i.text_with_ws

            print("Did you mean: ", updatedQuery)
            doc._.set("outcome_spellCheck",updatedQuery)
            print("Original text:", query)
        return updatedQuery, doc

    def misspellIdentify(self, query=""):
        """To identify misspelled words from the query

        At present, All the following criteria should be met for word to be misspelled
        1. Should not in our vocab
        2. should not be a Person
        3. Should not be a number


        Keyword Arguments:
            query {str} -- user query eg: "aa bb cc..." (default: {''})

        Returns:
            {tuple} -- returns `List[`Token`]` and `Doc`
        """

        doc = self.nlp(query)
        misspell = []
        for token in doc:
            if (
                (token.text.lower() not in self.vocab)
                and (token.ent_type_ != "PERSON")
                and (not token.like_num)
            ):

                misspell.append(token)

        if self.debug:
            print(misspell)
        return (misspell, doc)

    def candidateGenerator(self, doc, misspellings, top_n=5, query=""):
        """Returns Candidates for misspells

        This function is responsible for generating candidate list for misspell
        using BERT. The misspell is masked with a token and the model tries to 
        predict `n` candidates for the mask.

        Arguments:
            misspellings {List[`Token`]} -- Contains List of `Token` object types 
            from spacy to preserve meta information of the token 

        Keyword Arguments:
            top_n {int} -- Number of candidates to be generated (default: {5})
            query {User query} -- This is used for context pwered candidate generations.  (default: {''})

        Returns:
            Dict{`Token`:List[{str}]} -- Eg of return type {misspell-1:['candidate-1','candidate-2', ...],
                            misspell-2:['candidate-1','candidate-2'. ...]}
        """

        response = {}
        score = {}

        for token in misspellings:
            updatedQuery = ""
            for i in doc:
                if (i.i == token.i):
                    updatedQuery += self.mask + i.whitespace_
                else:
                    updatedQuery += i.text_with_ws
            if self.debug:
                print(
                    "For", "`" + token.text + "`", "updated query is:\n", updatedQuery
                )

            model_input = self.BertTokenizer.encode(updatedQuery, return_tensors="pt")
            mask_token_index = torch.where(
                model_input == self.BertTokenizer.mask_token_id
            )[1]
            token_logits = self.BertModel(model_input)[0]
            mask_token_logits = token_logits[0, mask_token_index, :]
            if self.debug:
                print("\nmask_token_logits:", mask_token_logits)

            token_probability = torch.nn.functional.softmax(mask_token_logits,dim=1)
            if self.debug: print("\ntoken_probability: ",token_probability)
            top_n_score, top_n_tokens = torch.topk(token_probability, top_n, dim=1)
            top_n_tokens = top_n_tokens[0].tolist()
            top_n_score = top_n_score[0].tolist()
            if self.debug:
                print("top_n_tokens:", top_n_tokens)
                print("token_score: ", top_n_score)

            if token not in response:
                response[token] = [
                    self.BertTokenizer.decode([candidateWord])
                    for candidateWord in top_n_tokens
                ]
                score[token]=[(self.BertTokenizer.decode([top_n_tokens[i]]),round(top_n_score[i],5)) for i in range(top_n)]

            # for candidate in top_5_tokens:
            # response[token].append(self.BertTokenizer.decode([candidate]))
            # print(updatedQuery.replace(self.mask, self.BertTokenizer.decode([candidate])))

            if self.debug: print("\nresponse: ",response,"\nscore: ",score)

        doc._.set("performed_spellCheck",True)
        doc._.set("score_spellCheck",score)
        

        return response

    def candidateRanking(self, misspellingsDict):
        """Ranking the candidates based on edit Distance

        At present using a library to calculate edit distance 
        between actual word and candidate words. Candidate word 
        for which edit distance is lowest is selected. If least 
        edit distance is same then word with higher probability 
        is selected by default

        Arguments:
            misspellingsDict {Dict{`Token`:List[{str}]}} -- 
            Orginal token is the key and candidate words are the values 

        Returns:
            Dict{`Token`:{str}} -- Eg of return type {misspell-1:'BEST-CANDIDATE'}
        """

        response = {}
        #         doc = self.nlp(query)
        for misspell in misspellingsDict:
            ## Init least_edit distance
            least_edit_dist = 100

            if self.debug:
                print("misspellingsDict[misspell]", misspellingsDict[misspell])
            for candidate in misspellingsDict[misspell]:
                edit_dist = editdistance.eval(misspell.text, candidate)
                if edit_dist < least_edit_dist:
                    least_edit_dist = edit_dist
                    response[misspell] = candidate

            if self.debug:
                print(response)
        return response



In [49]:
def timeLog(fnName, relativeTime):
    """For time log

    Arguments:
        fnName {str} -- function name to print
        relativeTime {datetime} -- previous date time for subtraction

    Returns:
        datetime -- datetime of current logging
    """

    timeNow = datetime.datetime.now()
    print(fnName, "took: ", timeNow - relativeTime)
    return datetime.datetime.now()



In [50]:
start=datetime.datetime.now()
checker = spellChecker(debug=True)
modelLoadTime = timeLog("Model Loading",start)

query = "Income was $9.4 milion compared to the prior year of $2.7 milion."

(updatedQuery, doc) = checker.check(query)
checkerTime = timeLog('Sentence Correction', modelLoadTime)


# misspellTokens = checker.misspellIdentify()
# misspellTime = timeLog("Misspell indetifying", modelLoadTime)

# candidate = checker.candidateGenerator(misspellTokens)
# candidateTime = timeLog("CandidateGeneration",misspellTime)

# answer = checker.candidateRanking(candidate)
# timeLog("ranking",candidateTime)
# for key in answer:
#     print('wrong spelling: ','`'+key.text+'`',"-- best candidate:", '`'+answer[key]+'`')
# print(answer)


Model Loading took:  0:00:08.901431
[milion, milion]
Misspell identification:  took:  0:00:00.005834
For `milion` updated query is:
 Income was $9.4 [MASK] compared to the prior year of $2.7 milion.

mask_token_logits: tensor([[-5.2289, -5.2235, -5.4641,  ..., -4.6217, -4.4137, -4.2567]],
       grad_fn=<IndexBackward>)

token_probability:  tensor([[4.2233e-09, 4.2460e-09, 3.3382e-09,  ..., 7.7504e-09, 9.5427e-09,
         1.1165e-08]], grad_fn=<SoftmaxBackward>)
top_n_tokens: [1550, 3775, 117, 27005, 16660]
token_score:  [0.5942245125770569, 0.2434934675693512, 0.08809225261211395, 0.018352238461375237, 0.008257790468633175]

response:  {milion: ['million', 'billion', ',', 'trillion', 'Million']} 
score:  {milion: [('million', 0.59422), ('billion', 0.24349), (',', 0.08809), ('trillion', 0.01835), ('Million', 0.00826)]}
For `milion` updated query is:
 Income was $9.4 milion compared to the prior year of $2.7 [MASK].

mask_token_logits: tensor([[-4.4341, -4.4109, -4.5952,  ..., -4.0313,

In [33]:
#             Doc.set_extension('contextual_spellCheck', default=True)
#             Doc.set_extension('performed_spellCheck', default=False)

#             # {originalToken-1:[suggestedToken-1,suggestedToken-2,..],
#             #  originalToken-2:[...]}
#             Doc.set_extension('suggestions_spellCheck', default=None)
#             Doc.set_extension('outcome_spellCheck', default="")
#             Doc.set_extension('score_spellCheck', default=None)

#             Span.set_extension('get_has_spellCheck', getter=self.span_require_spellCheck)
#             Span.set_extension('score_spellCheck', getter=self.span_score_spellCheck)

#             Token.set_extension('get_require_spellCheck', getter=self.token_require_spellCheck)
#             Token.set_extension('get_suggestion_spellCheck', getter=self.token_suggestion_spellCheck)
#             Token.set_extension('score_spellCheck', getter=self.token_score_spellCheck)
# #             Token.set_extension('score_spellCheck', default=1.0)


print("="*20,"Doc Extention Test", "="*20)
print(doc._.contextual_spellCheck)
print(doc._.performed_spellCheck)
print(doc._.suggestions_spellCheck)
print(doc._.outcome_spellCheck)
print(doc._.score_spellCheck)

print("="*20,"Token Extention Test", "="*20)
print(checker.token_require_spellCheck(doc[len(doc)-2]),doc[len(doc)-2].text)
print(doc[len(doc)-2]._.get_require_spellCheck)
print(doc[len(doc)-2]._.get_suggestion_spellCheck)
print(doc[len(doc)-2]._.score_spellCheck)


print("="*20,"Span Extention Test", "="*20)
print(checker.token_require_spellCheck(doc[len(doc)-2]),doc[len(doc)-2].text)
print(doc[len(doc)-7:len(doc)-1]._.get_has_spellCheck)
print(doc[len(doc)-7:len(doc)-1]._.score_spellCheck)


==================== Doc Extention Test ====================
True
True
{milion: ['million', 'billion', ',', 'trillion', 'Million'], milion: ['billion', 'million', 'trillion', '##M', 'Million']}
Income was $9.4 million compared to the prior year of $2.7 million.
{milion: [('million', 0.59422), ('billion', 0.24349), (',', 0.08809), ('trillion', 0.01835), ('Million', 0.00826)], milion: [('billion', 0.65934), ('million', 0.26185), ('trillion', 0.05391), ('##M', 0.0051), ('Million', 0.00425)]}
==================== Token Extention Test ====================
True milion
True
['billion', 'million', 'trillion', '##M', 'Million']
[[('billion', 0.65934), ('million', 0.26185), ('trillion', 0.05391), ('##M', 0.0051), ('Million', 0.00425)]]
==================== Span Extention Test ====================
True milion
True
[{prior: []}, {year: []}, {of: []}, {$: []}, {2.7: []}, {milion: [[('billion', 0.65934), ('million', 0.26185), ('trillion', 0.05391), ('##M', 0.0051), ('Million', 0.00425)]]}]


In [ ]:
print(doc[2].is_oov)



In [ ]:
a=[0.9035419225692749, 0.09310110658407211, 0.0012437802506610751, 0.0006837246473878622, 0.0005358955240808427]
a=[round(i,5) for i in a]
print(a)

b = [(a[i]+1,a[i]) for i in range(5)]
print(b)

## OLD CODE
***